In [1]:
import sys
sys.path = ["/Users/bernhard/Development/hcplot/"] + sys.path

In [2]:
from hcplot.highcharts import Figure, PlotOptions, DrillDown, DrillUpButton, SeriesOptions, Series, SeriesData, \
    Title, Legend, Chart, XAxis, YAxis, DataLabels, Position, SvgTheme, SvgButtonTheme, ButtonStates, Marker, \
    FillColor, LinearGradient, RadialGradient, GradientStop
from hcplot.highcharts.series import ColumnOptions, BubbleOptions


- Highsoft software products (www.highcharts.com) are not free for commercial and Governmental use.
  (see https://shop.highsoft.com/faq#Non-Commercial-0 for conditions for Non-commercial use).
- This product includes color specifications and designs developed by Cynthia Brewer
  http://colorbrewer.org
- D3 category colors and the interpolateRGB algorithm have been taken over from https://d3js.org/,
  Copyright 2010-2017 Mike Bostock



In [3]:
f = Figure(
    title=Title(text="Graph"), 
    chart=Chart(type_="column"), 
    xAxis=XAxis(type_="category"), 
    legend=Legend(enabled=False), 
    plotOptions=PlotOptions(       
        series=ColumnOptions(
            borderWidth=0, 
            dataLabels=DataLabels(enabled=True)
        )
    ), 
    series=[
        ColumnOptions(
            name="Things", 
            colorByPoint=True,
            data=[
                SeriesData(name='Dieren', y=5, drilldown='animals'),
                SeriesData(name='Fruit', y=2, drilldown='fruits'),
                SeriesData(name='Auto\'s', y=4)
            ]
        )
    ],
    drilldown=DrillDown(
        drillUpButton=DrillUpButton(
            relativeTo='spacingBox',
            position=Position(x=0, y=0),
            theme=SvgButtonTheme(
                fill='white', stroke_width=1, stroke='silver', r=0,
                states=ButtonStates(
                    hover=SvgTheme(fill='#a4edba'),
                    select=SvgTheme(stroke='#039',fill='#a4edba')
                )
            )
        ),
        series=[
            ColumnOptions(
                id_='animals',
                data=[
                    ['Katten', 4],
                    ['Honden', 2],
                    ['Koeien', 1],
                    ['Schapen', 2],
                    ['Varkens', 1]
                ]
            ), 
            ColumnOptions(
                id_='fruits',
                data=[
                    ['Appels', 4],
                    ['Sinaasappels', 2]
                ]
            )
        ]
    )
)

In [4]:
f.toDict()

{'chart': {'type': 'column'},
 'drilldown': {'drillUpButton': {'position': {'x': 0.0, 'y': 0.0},
   'relativeTo': 'spacingBox',
   'theme': {'fill': 'white',
    'r': 0,
    'states': {'hover': {'fill': '#a4edba'},
     'select': {'fill': '#a4edba', 'stroke': '#039'}},
    'stroke': 'silver',
    'strokewidth': 1}},
  'series': [{'data': [['Katten', 4],
     ['Honden', 2],
     ['Koeien', 1],
     ['Schapen', 2],
     ['Varkens', 1]],
    'id': 'animals'},
   {'data': [['Appels', 4], ['Sinaasappels', 2]], 'id': 'fruits'}]},
 'legend': {'enabled': False},
 'plotOptions': {'series': {'borderWidth': 0,
   'dataLabels': {'enabled': True}}},
 'series': [{'colorByPoint': True,
   'data': [{'drilldown': 'animals', 'name': 'Dieren', 'y': 5.0},
    {'drilldown': 'fruits', 'name': 'Fruit', 'y': 2.0},
    {'name': "Auto's", 'y': 4.0}],
   'name': 'Things'}],
 'title': {'text': 'Graph'},
 'xAxis': {'type': 'category'}}

In [5]:
data1 = [
    [9, 81, 63],
    [98, 5, 89],
    [51, 50, 73],
    [41, 22, 14],
    [58, 24, 20],
    [78, 37, 34],
    [55, 56, 53],
    [18, 45, 70],
    [42, 44, 28],
    [3, 52, 59],
    [31, 18, 97],
    [79, 91, 63],
    [93, 23, 23],
    [44, 83, 22]
]

data2 = [
    [42, 38, 20],
    [6, 18, 1],
    [1, 93, 55],
    [57, 2, 90],
    [80, 76, 22],
    [11, 74, 96],
    [88, 56, 10],
    [30, 47, 49],
    [57, 62, 98],
    [4, 16, 16],
    [46, 10, 11],
    [22, 87, 89],
    [57, 91, 82],
    [45, 15, 98]
]

f2 = Figure(
    chart = Chart(type_='bubble', plotBorderWidth=1, zoomType='xy'),
    title = Title (text='Highcharts bubbles with radial gradient fill'),
    xAxis = XAxis(gridLineWidth=1),
    yAxis = YAxis(startOnTick = False, endOnTick = False),
    series=[
        BubbleOptions(data=data1,
            marker=Marker(
                fillColor=FillColor (
                    radialGradient=RadialGradient(cx=0.4, cy=0.3, r=0.7),
                    stops=[
                        GradientStop(stop=0.0, color='rgba(255,0,0,0.5)'),
                        GradientStop(stop=1.0, color='rgba(0,0,255,0.5)')
                    ]
                )
            )
        ), 
        BubbleOptions(data=data2,
            marker=Marker(
                fillColor="rgba(0,255,255,0.5)"
            )
        )
    ]
)

In [6]:
f2.toDict()

{'chart': {'plotBorderWidth': 1, 'type': 'bubble', 'zoomType': 'xy'},
 'series': [{'data': [[9, 81, 63],
    [98, 5, 89],
    [51, 50, 73],
    [41, 22, 14],
    [58, 24, 20],
    [78, 37, 34],
    [55, 56, 53],
    [18, 45, 70],
    [42, 44, 28],
    [3, 52, 59],
    [31, 18, 97],
    [79, 91, 63],
    [93, 23, 23],
    [44, 83, 22]],
   'marker': {'fillColor': {'radialGradient': {'cx': 0.4, 'cy': 0.3, 'r': 0.7},
     'stops': [(0.0, 'rgba(255,0,0,0.500)'), (1.0, 'rgba(0,0,255,0.500)')]}}},
  {'data': [[42, 38, 20],
    [6, 18, 1],
    [1, 93, 55],
    [57, 2, 90],
    [80, 76, 22],
    [11, 74, 96],
    [88, 56, 10],
    [30, 47, 49],
    [57, 62, 98],
    [4, 16, 16],
    [46, 10, 11],
    [22, 87, 89],
    [57, 91, 82],
    [45, 15, 98]],
   'marker': {'fillColor': 'rgba(0,255,255,0.500)'}}],
 'title': {'text': 'Highcharts bubbles with radial gradient fill'},
 'xAxis': {'gridLineWidth': 1},
 'yAxis': {'endOnTick': False, 'startOnTick': False}}